**Импорт необходимых библиотек**

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from scipy import stats
import io
import matplotlib.pyplot as plt
import seaborn as sns

**Загрузим найденные датасеты**

In [2]:
from google.colab import files
uploaded = files.upload()

Saving SBER.csv to SBER.csv
Saving USD_RUB.csv to USD_RUB.csv
Saving ММВБ – Индекс Мосбиржи.csv to ММВБ – Индекс Мосбиржи.csv


In [3]:
sber_df = pd.read_csv(io.BytesIO(uploaded["SBER.csv"]))
sber_df.drop(['Объём', 'Изм. %', 'Макс.', 'Мин.'], axis = 1, inplace = True)
sber_df

,Дата,Цена,Откр.
0,14.05.2024,"318,05","315,00"
1,13.05.2024,"314,85","314,10"
2,10.05.2024,"313,49","311,50"
3,08.05.2024,"311,21","308,41"
4,07.05.2024,"308,22","306,23"
...,...,...,...
341,09.01.2023,"142,40","141,83"
342,06.01.2023,"141,40","141,39"
343,05.01.2023,"141,27","141,60"
344,04.01.2023,"141,43","141,85"


Удалили столбцы, которые не нужны для нашего анализа и модели. Теперь преобразуем колонку дата в формат datetime

In [4]:
 sber_df['Дата'] = pd.to_datetime(sber_df['Дата'])
 sber_df

<ipython-input-4-c3f6d1f9553f>:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  sber_df['Дата'] = pd.to_datetime(sber_df['Дата'])


,Дата,Цена,Откр.
0,2024-05-14,"318,05","315,00"
1,2024-05-13,"314,85","314,10"
2,2024-05-10,"313,49","311,50"
3,2024-05-08,"311,21","308,41"
4,2024-05-07,"308,22","306,23"
...,...,...,...
341,2023-01-09,"142,40","141,83"
342,2023-01-06,"141,40","141,39"
343,2023-01-05,"141,27","141,60"
344,2023-01-04,"141,43","141,85"


In [5]:
usd_rub_df = pd.read_csv(io.BytesIO(uploaded["USD_RUB.csv"]))
usd_rub_df.drop(['Объём', 'Изм. %', 'Макс.', 'Мин.'], axis = 1, inplace = True)
usd_rub_df

,Дата,Цена,Откр.
0,14.05.2024,"91,2528","91,3525"
1,13.05.2024,"91,3525","92,2550"
2,10.05.2024,"92,2550","91,9500"
3,08.05.2024,"91,9500","91,1075"
4,07.05.2024,"91,1075","91,3475"
...,...,...,...
341,09.01.2023,"69,9900","72,1225"
342,06.01.2023,"72,1225","72,1800"
343,05.01.2023,"72,1800","72,0000"
344,04.01.2023,"72,0000","71,1375"


In [6]:
usd_rub_df['Дата'] = pd.to_datetime(usd_rub_df['Дата'])
usd_rub_df

<ipython-input-6-0adb1b1901cd>:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  usd_rub_df['Дата'] = pd.to_datetime(usd_rub_df['Дата'])


,Дата,Цена,Откр.
0,2024-05-14,"91,2528","91,3525"
1,2024-05-13,"91,3525","92,2550"
2,2024-05-10,"92,2550","91,9500"
3,2024-05-08,"91,9500","91,1075"
4,2024-05-07,"91,1075","91,3475"
...,...,...,...
341,2023-01-09,"69,9900","72,1225"
342,2023-01-06,"72,1225","72,1800"
343,2023-01-05,"72,1800","72,0000"
344,2023-01-04,"72,0000","71,1375"


Дополним датасет данными по ключевой ставке и инфляции в РФ, которые мы решили спарсить с сайта ЦБ РФ,поскольку это их официальный источник

In [21]:
url = 'https://cbr.ru/hd_base/infl/?UniDbQuery.Posted=True&UniDbQuery.From=01.01.2023&UniDbQuery.To=14.05.2024'
resp = requests.get(url)
tree = BeautifulSoup(resp.content, 'html.parser')
table = tree.find('table')
df = pd.read_html(str(table))[0]


In [22]:
x = pd.DataFrame(columns = ['Дата','Ключевая ставка, % годовых','Инфляция, % г/г' ])

for n, i in enumerate(df['Дата']):
    if str(i)[:2] == '1.' or str(i)[:2] == '3.' or str(i)[:2] == '5.' or str(i)[:2] == '7.' or str(i)[:2] == '8.' or str(i)[:2] == '10':
        for el in range(1,32):
            x.loc[len(x.index )] = [f'{str(el)}.{str(i)}', df['Ключевая ставка, % годовых'].loc[n], df['Инфляция, % г/г'].loc[n]]
    elif str(i)[:2] == '4.' or str(i)[:2] == '6.' or str(i)[:2] == '9.' or str(i)[:2] == '11':
        for el in range(1,30):
            x.loc[len(x.index )] = [f'{str(el)}.{str(i)}', df['Ключевая ставка, % годовых'].loc[n], df['Инфляция, % г/г'].loc[n]]
    elif str(i)[:2] == '2.':
        for el in range(1,28):
            x.loc[len(x.index )] = [f'{str(el)}.{str(i)}', df['Ключевая ставка, % годовых'].loc[n], df['Инфляция, % г/г'].loc[n]]

x

,Дата,"Ключевая ставка, % годовых","Инфляция, % г/г"
0,1.4.2024,1600,784
1,2.4.2024,1600,784
2,3.4.2024,1600,784
3,4.4.2024,1600,784
4,5.4.2024,1600,784
...,...,...,...
442,27.1.2023,750,1177
443,28.1.2023,750,1177
444,29.1.2023,750,1177
445,30.1.2023,750,1177


In [23]:
x['Дата'] = pd.to_datetime(x['Дата'], format='%d.%m.%Y')
x['Дата'] = x['Дата'].dt.strftime('%Y-%m-%d') #преобразовали дату в нужный нам формат, потому что дальше нужно будет объединять все датафреймы именно по этому столбцу
x

,Дата,"Ключевая ставка, % годовых","Инфляция, % г/г"
0,2024-04-01,1600,784
1,2024-04-02,1600,784
2,2024-04-03,1600,784
3,2024-04-04,1600,784
4,2024-04-05,1600,784
...,...,...,...
442,2023-01-27,750,1177
443,2023-01-28,750,1177
444,2023-01-29,750,1177
445,2023-01-30,750,1177


In [24]:
mmvb_df = pd.read_csv(io.BytesIO(uploaded["ММВБ – Индекс Мосбиржи.csv"]))
mmvb_df.drop(['Объём', 'Изм. %', 'Макс.', 'Мин.'], axis = 1, inplace = True)
mmvb_df

,Дата,Цена,Откр.
0,16.05.2024,"3.485,72","3.480,48"
1,15.05.2024,"3.475,30","3.466,35"
2,14.05.2024,"3.464,77","3.465,83"
3,13.05.2024,"3.458,63","3.459,05"
4,10.05.2024,"3.449,87","3.438,50"
...,...,...,...
342,09.01.2023,"2.163,50","2.163,43"
343,06.01.2023,"2.156,39","2.157,32"
344,05.01.2023,"2.156,67","2.170,40"
345,04.01.2023,"2.168,42","2.171,54"


In [25]:
 mmvb_df['Дата'] = pd.to_datetime(mmvb_df['Дата'])

<ipython-input-25-591712f294e2>:1: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  mmvb_df['Дата'] = pd.to_datetime(mmvb_df['Дата'])


In [26]:
x = x.sort_values(by='Дата', ascending=False)
x['Дата'] = pd.to_datetime(x['Дата'])

In [29]:
sber_mmvb_df = pd.merge(sber_df, mmvb_df, on='Дата')
df1 = pd.merge(sber_mmvb_df, usd_rub_df, on = 'Дата')
df = pd.merge(df1, x, on='Дата')
df.columns = ['Дата', 'Цена_СБ', 'Откр.СБ', 'Цена_ДР', 'Откр.ДР', 'Цена_М', 'Откр.М', 'Ключевая ставка,%', 'Инфляция,%']
df

,Дата,Цена_СБ,Откр.СБ,Цена_ДР,Откр.ДР,Цена_М,Откр.М,"Ключевая ставка,%","Инфляция,%"
0,2024-04-29,"308,97","309,14","3.478,08","3.468,92","93,0000","92,3975",1600,784
1,2024-04-26,"309,00","308,50","3.449,77","3.446,39","91,7200","91,9375",1600,784
2,2024-04-25,"308,41","307,95","3.439,76","3.429,62","91,9375","92,2450",1600,784
3,2024-04-24,"307,94","307,50","3.428,93","3.438,45","92,2450","93,1275",1600,784
4,2024-04-23,"307,39","315,39","3.439,42","3.483,26","93,1275","93,4700",1600,784
...,...,...,...,...,...,...,...,...,...
305,2023-01-09,"142,40","141,83","2.163,50","2.163,43","69,9900","72,1225",750,1177
306,2023-01-06,"141,40","141,39","2.156,39","2.157,32","72,1225","72,1800",750,1177
307,2023-01-05,"141,27","141,60","2.156,67","2.170,40","72,1800","72,0000",750,1177
308,2023-01-04,"141,43","141,85","2.168,42","2.171,54","72,0000","71,1375",750,1177


In [ ]:
df.isnull().sum()

Дата                 0
Цена_СБ              0
Откр.СБ              0
Цена_ДР              0
Откр.ДР              0
Рабочий день         0
Цена_М               0
Откр.М               0
Ключевая ставка,%    0
Инфляция,%           0
dtype: int64

Пропусков нет, для каждой даты есть данные по каждой из переменных.

In [30]:
df['Ключевая ставка,%'] = df['Ключевая ставка,%'].div(100)
df['Инфляция,%'] = df['Инфляция,%'].div(100)
df

,Дата,Цена_СБ,Откр.СБ,Цена_ДР,Откр.ДР,Цена_М,Откр.М,"Ключевая ставка,%","Инфляция,%"
0,2024-04-29,"308,97","309,14","3.478,08","3.468,92","93,0000","92,3975",16.0,7.84
1,2024-04-26,"309,00","308,50","3.449,77","3.446,39","91,7200","91,9375",16.0,7.84
2,2024-04-25,"308,41","307,95","3.439,76","3.429,62","91,9375","92,2450",16.0,7.84
3,2024-04-24,"307,94","307,50","3.428,93","3.438,45","92,2450","93,1275",16.0,7.84
4,2024-04-23,"307,39","315,39","3.439,42","3.483,26","93,1275","93,4700",16.0,7.84
...,...,...,...,...,...,...,...,...,...
305,2023-01-09,"142,40","141,83","2.163,50","2.163,43","69,9900","72,1225",7.5,11.77
306,2023-01-06,"141,40","141,39","2.156,39","2.157,32","72,1225","72,1800",7.5,11.77
307,2023-01-05,"141,27","141,60","2.156,67","2.170,40","72,1800","72,0000",7.5,11.77
308,2023-01-04,"141,43","141,85","2.168,42","2.171,54","72,0000","71,1375",7.5,11.77


Преобразовали столбцы со значениями ключевой ставки и инфляции, чтобы в ячейках действительно отображались процентные значения

In [31]:
df.dtypes

Дата                 datetime64[ns]
Цена_СБ                      object
Откр.СБ                      object
Цена_ДР                      object
Откр.ДР                      object
Цена_М                       object
Откр.М                       object
Ключевая ставка,%           float64
Инфляция,%                  float64
dtype: object

Видим, что большинство ячеек сейчас имеют формат object, но для дальнейшего анализа нам необходимы численные значения, поэтому напишем функцию, которая позволит нам получить float.

In [32]:
def correct_values(value: str):
    value = str(value)
    corrected_value = value.replace('.','').replace(',','.')
    corrected_value = float(corrected_value)
    return corrected_value

In [33]:
df['Цена_СБ'] = df['Цена_СБ'].map(correct_values,na_action = 'ignore')
df['Откр.СБ'] = df['Откр.СБ'].map(correct_values,na_action = 'ignore')
df['Цена_ДР'] = df['Цена_ДР'].map(correct_values,na_action = 'ignore')
df['Откр.ДР'] = df['Откр.ДР'].map(correct_values,na_action = 'ignore')
df['Цена_М'] = df['Цена_М'].map(correct_values,na_action = 'ignore')
df['Откр.М'] = df['Откр.М'].map(correct_values,na_action = 'ignore')
df

,Дата,Цена_СБ,Откр.СБ,Цена_ДР,Откр.ДР,Цена_М,Откр.М,"Ключевая ставка,%","Инфляция,%"
0,2024-04-29,308.97,309.14,3478.08,3468.92,93.0000,92.3975,16.0,7.84
1,2024-04-26,309.00,308.50,3449.77,3446.39,91.7200,91.9375,16.0,7.84
2,2024-04-25,308.41,307.95,3439.76,3429.62,91.9375,92.2450,16.0,7.84
3,2024-04-24,307.94,307.50,3428.93,3438.45,92.2450,93.1275,16.0,7.84
4,2024-04-23,307.39,315.39,3439.42,3483.26,93.1275,93.4700,16.0,7.84
...,...,...,...,...,...,...,...,...,...
305,2023-01-09,142.40,141.83,2163.50,2163.43,69.9900,72.1225,7.5,11.77
306,2023-01-06,141.40,141.39,2156.39,2157.32,72.1225,72.1800,7.5,11.77
307,2023-01-05,141.27,141.60,2156.67,2170.40,72.1800,72.0000,7.5,11.77
308,2023-01-04,141.43,141.85,2168.42,2171.54,72.0000,71.1375,7.5,11.77


Датасет для дальнейшего анализа готов